In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import shap
import ipywidgets
import torch
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
from pathlib import Path
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier, Pool
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, roc_auc_score,confusion_matrix, 
                            classification_report)

In [ ]:
#Конкатенация собранных .csv файлов в один
data_direction = Path("/Users/sasha/Documents/Kickstarter2024/")
df = pd.concat([pd.read_csv(f) for f in data_direction.glob("*.csv")], ignore_index=True)
df.to_csv("/Users/Sasha/Documents/dataframe.csv", index=False)

In [ ]:
#Чтение файла
path = '/Users/sasha/Documents/dataframe.csv'
data = pd.read_csv(path)
data.info()
data.columns.tolist()

In [ ]:
df_clear = data.drop(['backers_count', 'converted_pledged_amount', 'country_displayable_name',
                      'creator', 'currency', 'currency_symbol', 'currency_trailing_code',
                      'current_currency', 'disable_communication', 'fx_rate',
                      'id', 'is_disliked', 'is_liked', 'is_launched',
                      'is_in_post_campaign_pledging_phase', 'is_starrable', 'name',
                      'percent_funded', 'photo', 'pledged', 'profile', 'spotlight', 
                      'slug', 'source_url', 'state_changed_at', 'static_usd_rate', 'urls',
                       'usd_exchange_rate', 'usd_pledged', 'usd_type'], axis=1)
df_clear.columns.tolist()


In [ ]:
df_clear.at[20,'location']
df_clear = df_clear.rename(columns={'blurb': 'Текст', 'category': 'Категория',
                                    'country': 'Страна', 'goal': 'Цель',
                                    'location': 'Город', 'prelaunch_activated': 'РаннийЗапуск',
                                    'staff_pick': 'ВыборПлощадки', 'state': 'Результат',
                                    'video': 'Видео'})


In [ ]:
#процентное соотношение пропущенных значений
df_nans = (df_clear.isnull().sum() / len(df_clear)) * 100

#удаление столбцов без пропущенных значений, сортировка
df_nans = df_nans.drop(df_nans[df_nans == 0].index).sort_values(ascending=False)

f, ax = plt.subplots(figsize=(12, 8))
plt.xticks(rotation=90)
sns.barplot(x=df_nans.index, y=df_nans)
ax.set(title='Процент пропущенных данных на признак', ylabel='Процент пропущенных данных')
plt.show()

In [141]:
df_clear['Видео'] = df_clear['Видео'].fillna(0)
df_clear = df_clear.dropna()

In [ ]:
def extract_parent_name(cell):
    try:
        data1 = ast.literal_eval(cell) if isinstance(cell, str) else cell
        return data1.get('parent_name')
    except (ValueError, SyntaxError, AttributeError):
        import re
        match = re.search(r"'parent_name'\s*:\s*'([^']+)'", str(cell))
        if not match:
            match = re.search(r'"parent_name"\s*:\s*"([^"]+)"', str(cell))
        return match.group(1) if match else None

df_clear['Категория'] = df_clear['Категория'].apply(extract_parent_name)
df_clear = df_clear.dropna()

In [ ]:
def extract_location_name(cell):
    try:
        data2 = ast.literal_eval(cell) if isinstance(cell, str) else cell
        return data2.get('name')
    except (ValueError, SyntaxError, AttributeError):
        import re
        match = re.search(r"'name'\s*:\s*'([^']+)'", str(cell)) or \
               re.search(r'"name"\s*:\s*"([^"]+)"', str(cell))
        return match.group(1) if match else None

df_clear['Город'] = df_clear['Город'].apply(extract_location_name)
df_clear['Видео'] = np.where(df_clear['Видео'] == 0, 0, 1)

In [ ]:
display(df_clear.iloc[[183540]].T)
df_clear['Длительность'] = (df_clear['deadline'] - df_clear['launched_at']) // 86400
df_clear['ДлительностьПодготовки'] = (df_clear['launched_at'] - df_clear['created_at']) // 86400
df_clear = df_clear.drop(['launched_at', 'created_at', 'deadline'], axis=1)


In [147]:
df_clear[['РаннийЗапуск', 'ВыборПлощадки']] = df_clear[['РаннийЗапуск', 'ВыборПлощадки']].astype(int)

In [ ]:
df_clear['Результат'] = df_clear['Результат'].replace({
    'successful': 1,
    'failed': 0,
    'canceled': 0,
    'submitted': 2,
    'live': 2,
    'started': 2,
    'suspended': 2
    })
df_clear.info()

In [ ]:
state_counts = df_clear['Результат'].value_counts()
state_percentage = df_clear['Результат'].value_counts(normalize=True) * 100

state_stats = pd.DataFrame({
    'Count': state_counts,
    'Percentage': state_percentage
})

print(state_stats)

In [150]:
df_clear = df_clear[df_clear['Результат'] != 2]

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(np.log1p(df_clear['Цель']), kde=False, bins=30)
plt.title('Распределение логарифма целевой суммы сбора')
plt.xlabel('Целевая сумма сбора')
plt.ylabel('Частота')
plt.show()

In [ ]:
numeric_features = ['Цель', 'Длительность', 'ДлительностьПодготовки']
success_stats = df_clear.groupby('Результат')[numeric_features].agg(['mean', 'median'])
print("\nСравнение средних и медиан:")
print(success_stats)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

#ящик с усами для каждого признака
for i, feature in enumerate(numeric_features):
    sns.boxplot(x='Результат', y=feature, data=df_clear, ax=axes[i], showfliers=False)
    axes[i].set_title(f'Распределение признака {feature}')
    axes[i].set_xlabel('Успех кампании')
    axes[i].set_ylabel(feature)
    
    #Аннотации со средними и медианами
    for status in [0, 1]:
        median = df_clear[df_clear['Результат'] == status][feature].median()
        mean = df_clear[df_clear['Результат'] == status][feature].mean()
        axes[i].text(status, median*1.05, f'Мед: {median:.1f}\nСр: {mean:.1f}', 
                    ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df_clear, x='Длительность', hue='Результат', bins=30, kde=False, element='step')
plt.title('Распределение длительности кампании')
plt.xlabel('Длительность (дни)')
plt.ylabel('Частота')
plt.show()

In [ ]:
category_success = df_clear.groupby(['Категория', 'Результат']).size().unstack().fillna(0)
category_success['success_rate'] = category_success[1] / (category_success[0] + category_success[1])
category_success = category_success.sort_values('success_rate', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=category_success.index, y='success_rate', data=category_success, palette='viridis')
plt.title('Успешность по категориям проектов')
plt.xlabel('Категория')
plt.ylabel('Доля успешных кампаний')
plt.xticks(rotation=45)
plt.show()

In [ ]:
country_success = df_clear.groupby('Страна')['Результат'].agg(['mean', 'count'])
country_success = country_success[country_success['count'] > 100]
country_success = country_success.sort_values('mean', ascending=False).head(30)

plt.figure(figsize=(12, 6))
sns.barplot(x=country_success.index, y='mean', data=country_success, palette='coolwarm')
plt.title('Распределение успеха кампаний по странам')
plt.xlabel('Страна')
plt.ylabel('Доля успешных кампаний')
plt.xticks(rotation=45)
plt.show()

In [ ]:
binary_features = ['Видео', 'ВыборПлощадки']

plt.figure(figsize=(15, 5))
for i, feature in enumerate(binary_features, 1):
    plt.subplot(1, 3, i)
    sns.barplot(x=feature, y='Результат', data=df_clear, errorbar=None)
    plt.title(f'Успешность кампаний по фактору {feature}')
    plt.ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
numerical_df = df_clear[['Результат', 'Цель', 'Длительность', 'ДлительностьПодготовки', ] + binary_features]
plt.figure(figsize=(10, 8))
sns.heatmap(numerical_df.corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Матрица корреляций')
plt.show()

In [ ]:
df_clear['Текст'] = df_clear['Текст'].fillna('')

tfidf = TfidfVectorizer(
    max_features=1000,  # Ограничим количество слов для интерпретируемости
    min_df=5,          # Слово должно встречаться минимум в 5 кампаниях
    max_df=0.8,        # Исключаем слова, встречающиеся в >80% кампаний
    stop_words='english',  # Удаляем стоп-слова
    ngram_range=(1, 2)    # Учитываем отдельные слова и биграммы
)

# Применяем TF-IDF к тексту
tfidf_features = tfidf.fit_transform(df_clear['Текст'].fillna('').astype(str))

# Преобразуем в DataFrame
tfidf_df = pd.DataFrame(
    tfidf_features.toarray(),
    columns=tfidf.get_feature_names_out())

# 2. Объединяем с исходными данными
# Удаляем исходный текстовый признак
df_without_text = df_clear.drop('Текст', axis=1)

# Объединяем с TF-IDF фичами
df_clear = pd.concat([df_without_text, tfidf_df], axis=1)
df_clear = df_clear.dropna()


In [161]:
#Предобработка данных
X = df_clear.drop('Результат', axis=1)
y = df_clear['Результат']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [162]:
#Распределение признаков по типам

cat_features = ['Категория', 'Страна', 'Город']

train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=cat_features,
    feature_names=list(X_train.columns))

test_pool = Pool(
    data=X_test,
    label=y_test,
    cat_features=cat_features,
    feature_names=list(X_test.columns))


In [163]:
#Инициализация
model = CatBoostClassifier(
    auto_class_weights='Balanced',
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=100)

In [ ]:
#Обучение модели
model.fit(
    train_pool,
    eval_set=test_pool,
    plot=True)

In [ ]:
#Оценка качества
y_pred = model.predict(test_pool)
y_pred_proba = model.predict_proba(test_pool)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nMetrics:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

#Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Провал (0)', 'Успех (1)'], 
            yticklabels=['Провал (0)', 'Успех (1)'])
plt.title('Матрица ошибок')
plt.ylabel('В действительности')
plt.xlabel('Предсказано')
plt.show()

In [ ]:
feature_importance = model.get_feature_importance()

word_importance = pd.DataFrame({
    'word': X_train.columns[len(df_without_text.columns)-1:],  # Только словарные фичи
    'importance': feature_importance[len(df_without_text.columns)-1:]
})

top_words = word_importance.sort_values('importance', ascending=False).head(30)

plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='word', data=top_words, palette='viridis')
plt.title('Топ-40 слов, наиболее влияющих на успех кампаний')
plt.xlabel('Важность признака')
plt.ylabel('Слово')
plt.show()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)
model.eval()

In [ ]:
def get_embeddings(texts, batch_size=64, show_progress=True):

    embeddings = []
    
    
    iter_range = range(0, len(texts), batch_size)
    if show_progress:
        iter_range = tqdm(iter_range, desc="Processing batches")
    
    for i in iter_range:
        batch = texts[i:i + batch_size]
        
        encoded = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors='pt'
        ).to(device)
        
        with torch.no_grad():
            outputs = model(**encoded)
        
        # Извлечение последнего скрытого слоя и усреднение по токенам (mean pooling)
        last_hidden_state = outputs.last_hidden_state
        attention_mask = encoded['attention_mask']
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        batch_embeddings = sum_embeddings / sum_mask
        
        embeddings.append(batch_embeddings.cpu().numpy())
    
    return np.concatenate(embeddings, axis=0)


In [ ]:
texts = df_clear['Текст'].astype(str).tolist()
embeddings = get_embeddings(texts)

In [95]:
np.save('text_embeddings.npy', embeddings)

In [96]:
bert_columns = [f'bert_{i}' for i in range(embeddings.shape[1])]
bert_df = pd.DataFrame(embeddings, columns=bert_columns)

In [ ]:
df_without_text = df_clear.drop('Текст', axis=1)
final_df = pd.concat([df_without_text, bert_df], axis=1)
final_df = final_df.dropna()

In [ ]:
#Предобработка данных
X = final_df.drop('Результат', axis=1)
y = final_df['Результат']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
cat_features = ['Категория', 'Страна', 'Город']

train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=cat_features,
    feature_names=list(X_train.columns))

test_pool = Pool(
    data=X_test,
    label=y_test,
    cat_features=cat_features,
    feature_names=list(X_test.columns))


In [ ]:
#обучение с эмбеддингами

model = CatBoostClassifier(
    auto_class_weights='Balanced',
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=100
)

model.fit(
    train_pool,
    eval_set=test_pool,
    plot=True)

In [ ]:
#оценка модели
y_pred = model.predict(test_pool)
y_pred_proba = model.predict_proba(test_pool)[:, 1]
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nMetrics:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

#Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Провал (0)', 'Успех (1)'], 
            yticklabels=['Провал (0)', 'Успех (1)'])
plt.title('Матрица ошибок')
plt.ylabel('В действительности')
plt.xlabel('Предсказано')
plt.show()